In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm

# import sklearn.preprocessing

In [ ]:
os.chdir("/content/drive/MyDrive/Brain Dead IPL Dataset")

In [ ]:
deliveries = pd.read_csv("deliveries.csv")
matches = pd.read_csv("matches.csv")

In [ ]:
players = pd.read_csv("player_stats.csv")
teams = pd.read_csv("team_stat.csv")
seasons = pd.read_csv("seasonal_stat.csv")
top_bowlers_stats = pd.read_csv("top_bowlers_stats.csv")

In [ ]:
deliveries.info(), matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260920 entries, 0 to 260919
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   match_id          260920 non-null  int64 
 1   inning            260920 non-null  int64 
 2   batting_team      260920 non-null  object
 3   bowling_team      260920 non-null  object
 4   over              260920 non-null  int64 
 5   ball              260920 non-null  int64 
 6   batter            260920 non-null  object
 7   bowler            260920 non-null  object
 8   non_striker       260920 non-null  object
 9   batsman_runs      260920 non-null  int64 
 10  extra_runs        260920 non-null  int64 
 11  total_runs        260920 non-null  int64 
 12  extras_type       14125 non-null   object
 13  is_wicket         260920 non-null  int64 
 14  player_dismissed  12950 non-null   object
 15  dismissal_kind    12950 non-null   object
 16  fielder           9354 non-null    obj

(None, None)

In [ ]:
print(deliveries.head(10))
print(matches.head(10))

   match_id  inning           batting_team                 bowling_team  over  \
0    335982       1  Kolkata Knight Riders  Royal Challengers Bangalore     0   
1    335982       1  Kolkata Knight Riders  Royal Challengers Bangalore     0   
2    335982       1  Kolkata Knight Riders  Royal Challengers Bangalore     0   
3    335982       1  Kolkata Knight Riders  Royal Challengers Bangalore     0   
4    335982       1  Kolkata Knight Riders  Royal Challengers Bangalore     0   
5    335982       1  Kolkata Knight Riders  Royal Challengers Bangalore     0   
6    335982       1  Kolkata Knight Riders  Royal Challengers Bangalore     0   
7    335982       1  Kolkata Knight Riders  Royal Challengers Bangalore     1   
8    335982       1  Kolkata Knight Riders  Royal Challengers Bangalore     1   
9    335982       1  Kolkata Knight Riders  Royal Challengers Bangalore     1   

   ball       batter   bowler  non_striker  batsman_runs  extra_runs  \
0     1   SC Ganguly  P Kumar  BB Mc

In [ ]:
deliveries.isnull().sum(),matches.isnull().sum()

(match_id                 0
 inning                   0
 batting_team             0
 bowling_team             0
 over                     0
 ball                     0
 batter                   0
 bowler                   0
 non_striker              0
 batsman_runs             0
 extra_runs               0
 total_runs               0
 extras_type         246795
 is_wicket                0
 player_dismissed    247970
 dismissal_kind      247970
 fielder             251566
 dtype: int64,
 id                    0
 season                0
 city                 51
 date                  0
 match_type            0
 player_of_match       5
 venue                 0
 team1                 0
 team2                 0
 toss_winner           0
 toss_decision         0
 winner                5
 result                0
 result_margin        19
 target_runs           3
 target_overs          3
 super_over            0
 method             1074
 umpire1               0
 umpire2               0
 dtype: i

In [ ]:
deliveries['extras_type'].unique()

array(['legbyes', nan, 'wides', 'byes', 'noballs', 'penalty'],
      dtype=object)

In [ ]:
from collections import defaultdict

player_stats = defaultdict(lambda: {
    'match_count': 0, 'total_run_bat': 0, 'balls_faced': 0, 'total_wicket': 0, 'total_run_bowl': 0,
    'balling_over': 0.0, '6_count': 0, '4_count': 0, '2_count': 0, '1_count': 0,
    'batting_average': 0.0, 'strike_rate': 0.0, 'bowling_economy': 0, 'highest_run': 0
})

player_dismissals = defaultdict(int)
player_highest_score = defaultdict(int)
previous_match = defaultdict(lambda: None)
player_match_runs = defaultdict(lambda: defaultdict(int))
player_match_played = defaultdict(set)

for _, row in tqdm(deliveries.iterrows(), total=deliveries.shape[0]):
    match_id = row['match_id']
    batter = row['batter']
    bowler = row['bowler']
    runs = row['batsman_runs']
    run = row['total_runs']
    is_wicket = row['is_wicket']
    extra_type = row['extras_type']

    player_match_runs[batter][match_id] += runs

    if match_id not in player_match_played[batter]:
        player_stats[batter]['match_count'] += 1
        player_match_played[batter].add(match_id)

    player_stats[batter]['total_run_bat'] += runs
    player_stats[bowler]['total_run_bowl'] += run

    if runs == 6:
        player_stats[batter]['6_count'] += 1
    elif runs == 4:
        player_stats[batter]['4_count'] += 1
    elif runs == 2:
        player_stats[batter]['2_count'] += 1
    elif runs == 1:
        player_stats[batter]['1_count'] += 1

    if pd.isna(extra_type) or extra_type not in ['wides', 'noballs']:
        player_stats[batter]['balls_faced'] += 1

    if is_wicket and not pd.isna(row['player_dismissed']):
        player_dismissals[batter] += 1

    if pd.isna(extra_type) or extra_type not in ['wides', 'noballs']:
        player_stats[bowler]['balling_over'] += 0.1
        if player_stats[bowler]['balling_over'] % 1.0 >= 0.6:
            player_stats[bowler]['balling_over'] = round(player_stats[bowler]['balling_over'] + 0.4, 1)


    if is_wicket and not pd.isna(row['player_dismissed']):
        player_stats[bowler]['total_wicket'] += 1


for player, stats in player_stats.items():
    stats['strike_rate'] = (stats['total_run_bat'] * 100) / stats['balls_faced'] if stats['balls_faced'] > 0 else 0
    stats['bowling_economy'] = stats['total_run_bowl'] / stats['balling_over'] if stats['balling_over'] > 0 else 0

for player in player_stats.keys():
    # Highest individual score in a single match
    player_stats[player]['highest_run'] = max(player_match_runs[player].values(), default=0)

    # Batting average: total runs / dismissals (None if never dismissed)
    outs = player_dismissals[player]
    player_stats[player]['batting_average'] = player_stats[player]['total_run_bat'] / outs if outs > 0 else None

players = pd.DataFrame.from_dict(player_stats, orient='index').reset_index()
players.rename(columns={'index': 'player'}, inplace=True)

players.head(20)

100%|██████████| 260920/260920 [00:12<00:00, 20321.06it/s]


,player,match_count,total_run_bat,balls_faced,total_wicket,total_run_bowl,balling_over,6_count,4_count,2_count,1_count,batting_average,strike_rate,bowling_economy,highest_run
0,SC Ganguly,56,1349,1257,12,370,46.0,42,137,54,427,24.981481,107.319014,8.043478,91
1,P Kumar,57,340,312,102,3342,420.4,17,22,24,102,10.000000,108.974359,7.949572,34
2,BB McCullum,109,2882,2176,0,0,0.0,130,293,103,715,27.188679,132.444853,0.000000,158
3,Z Khan,27,117,140,119,2860,366.4,2,11,7,47,7.800000,83.571429,7.805677,23
4,AA Noffke,1,9,10,1,41,4.0,0,1,0,5,9.000000,90.000000,10.250000,9
5,RT Ponting,9,91,128,0,0,0.0,2,5,6,47,11.375000,71.093750,0.000000,28
6,JH Kallis,95,2427,2213,74,2348,290.2,44,255,113,888,27.269663,109.670131,8.090972,89
7,SB Joshi,2,6,14,1,82,9.1,0,0,0,6,3.000000,42.857143,9.010989,3
8,DJ Hussey,61,1322,1070,10,485,52.5,60,90,72,452,26.979592,123.551402,9.238095,71
9,CL White,45,971,757,1,86,6.6,38,76,49,335,27.742857,128.269485,13.030303,78


In [ ]:
players[players['player'] == 'MS Dhoni']

,player,match_count,total_run_bat,balls_faced,total_wicket,total_run_bowl,balling_over,6_count,4_count,2_count,1_count,batting_average,strike_rate,bowling_economy,highest_run
27,MS Dhoni,228,5243,3790,0,0,0.0,252,363,340,1554,35.187919,138.337731,0.0,84


In [ ]:
players[players['player'] == 'J Yadav']

,player,match_count,total_run_bat,balls_faced,total_wicket,total_run_bowl,balling_over,6_count,4_count,2_count,1_count,batting_average,strike_rate,bowling_economy,highest_run
433,J Yadav,4,40,36,9,447,64.6,1,2,5,16,20.0,111.111111,6.919505,23


In [ ]:
players['MOTM_count'] = 0  # Initialize MOTM_count to 0 for all players

for index, row in matches.iterrows():
  player_of_match = row['player_of_match']
  if player_of_match in players['player'].values:
    players.loc[players['player'] == player_of_match, 'MOTM_count'] += 1

players.to_csv('player_stats.csv', index=False)
players.head(20)


,player,match_count,total_run_bat,balls_faced,total_wicket,total_run_bowl,balling_over,6_count,4_count,2_count,1_count,batting_average,strike_rate,bowling_economy,highest_run,MOTM_count
0,SC Ganguly,56,1349,1257,12,370,46.0,42,137,54,427,24.981481,107.319014,8.043478,91,5
1,P Kumar,57,340,312,102,3342,420.4,17,22,24,102,10.000000,108.974359,7.949572,34,2
2,BB McCullum,109,2882,2176,0,0,0.0,130,293,103,715,27.188679,132.444853,0.000000,158,5
3,Z Khan,27,117,140,119,2860,366.4,2,11,7,47,7.800000,83.571429,7.805677,23,1
4,AA Noffke,1,9,10,1,41,4.0,0,1,0,5,9.000000,90.000000,10.250000,9,0
5,RT Ponting,9,91,128,0,0,0.0,2,5,6,47,11.375000,71.093750,0.000000,28,0
6,JH Kallis,95,2427,2213,74,2348,290.2,44,255,113,888,27.269663,109.670131,8.090972,89,10
7,SB Joshi,2,6,14,1,82,9.1,0,0,0,6,3.000000,42.857143,9.010989,3,0
8,DJ Hussey,61,1322,1070,10,485,52.5,60,90,72,452,26.979592,123.551402,9.238095,71,2
9,CL White,45,971,757,1,86,6.6,38,76,49,335,27.742857,128.269485,13.030303,78,1


In [ ]:
players.sort_values(by='MOTM_count', ascending=False).head(10)

,player,match_count,total_run_bat,balls_faced,total_wicket,total_run_bowl,balling_over,6_count,4_count,2_count,1_count,batting_average,strike_rate,bowling_economy,highest_run,MOTM_count
127,AB de Villiers,170,5181,3400,0,0,0.0,253,414,268,1420,41.448000,152.382353,0.000000,133,25
174,CH Gayle,141,4997,3331,19,755,92.2,359,408,92,1018,39.039062,150.015011,8.188720,175,22
78,RG Sharma,251,6630,5039,16,462,56.3,281,599,263,1996,28.577586,131.573725,8.206039,109,19
200,DA Warner,184,6567,4682,0,2,0.1,236,663,370,1682,40.042683,140.260572,20.000000,126,18
15,V Kohli,244,8014,6051,5,371,41.5,273,708,445,2591,36.761468,132.440919,8.939759,113,18
27,MS Dhoni,228,5243,3790,0,0,0.0,252,363,340,1554,35.187919,138.337731,0.000000,84,17
45,SR Watson,141,3880,2801,107,2742,338.1,190,377,135,933,30.793651,138.521956,8.110027,117,16
50,RA Jadeja,180,2959,2267,169,4917,638.1,107,215,213,1013,25.956140,130.524923,7.705689,62,16
43,YK Pathan,153,3222,2233,46,1443,191.1,161,263,156,877,29.027027,144.290193,7.551020,100,16
342,AD Russell,104,2488,1407,125,2691,282.5,209,171,60,430,29.619048,176.830135,9.525664,88,15


In [ ]:
matches["team1"].unique()

array(['Royal Challengers Bangalore', 'Kings XI Punjab',
       'Delhi Daredevils', 'Mumbai Indians', 'Kolkata Knight Riders',
       'Rajasthan Royals', 'Deccan Chargers', 'Chennai Super Kings',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Gujarat Lions', 'Rising Pune Supergiants',
       'Rising Pune Supergiant', 'Delhi Capitals', 'Punjab Kings',
       'Lucknow Super Giants', 'Gujarat Titans',
       'Royal Challengers Bengaluru'], dtype=object)

In [ ]:
deliveries['batting_team'].unique()

array(['Kolkata Knight Riders', 'Royal Challengers Bangalore',
       'Chennai Super Kings', 'Kings XI Punjab', 'Rajasthan Royals',
       'Delhi Daredevils', 'Mumbai Indians', 'Deccan Chargers',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Rising Pune Supergiants', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Delhi Capitals', 'Punjab Kings',
       'Lucknow Super Giants', 'Gujarat Titans',
       'Royal Challengers Bengaluru'], dtype=object)

In [ ]:
# Replace inconsistent team names in matches
matches['team1'] = matches['team1'].replace('Delhi Daredevils', 'Delhi Capitals')
matches['team2'] = matches['team2'].replace('Delhi Daredevils', 'Delhi Capitals')
matches['toss_winner'] = matches['toss_winner'].replace('Delhi Daredevils', 'Delhi Capitals')
matches['winner'] = matches['winner'].replace('Delhi Daredevils', 'Delhi Capitals')
matches['team1'] = matches['team1'].replace('Deccan Chargers', 'Sunrisers Hyderabad')
matches['team2'] = matches['team2'].replace('Deccan Chargers', 'Sunrisers Hyderabad')
matches['toss_winner'] = matches['toss_winner'].replace('Deccan Chargers', 'Sunrisers Hyderabad')
matches['winner'] = matches['winner'].replace('Deccan Chargers', 'Sunrisers Hyderabad')
matches['team1'] = matches['team1'].replace('Kings XI Punjab', 'Punjab Kings')
matches['team2'] = matches['team2'].replace('Kings XI Punjab', 'Punjab Kings')
matches['toss_winner'] = matches['toss_winner'].replace('Kings XI Punjab', 'Punjab Kings')
matches['winner'] = matches['winner'].replace('Kings XI Punjab', 'Punjab Kings')
matches['team1'] = matches['team1'].replace('Rising Pune Supergiants', 'Rising Pune Supergiant')
matches['team2'] = matches['team2'].replace('Rising Pune Supergiants', 'Rising Pune Supergiant')
matches['toss_winner'] = matches['toss_winner'].replace('Rising Pune Supergiants', 'Rising Pune Supergiant')
matches['winner'] = matches['winner'].replace('Rising Pune Supergiants', 'Rising Pune Supergiant')
matches['team1'] = matches['team1'].replace('Royal Challengers Bangalore', 'Royal Challengers Bengaluru')
matches['team2'] = matches['team2'].replace('Royal Challengers Bangalore', 'Royal Challengers Bengaluru')
matches['toss_winner'] = matches['toss_winner'].replace('Royal Challengers Bangalore', 'Royal Challengers Bengaluru')
matches['winner'] = matches['winner'].replace('Royal Challengers Bangalore', 'Royal Challengers Bengaluru')


# Replace inconsistent team names in deliveries
deliveries['batting_team'] = deliveries['batting_team'].replace('Delhi Daredevils', 'Delhi Capitals')
deliveries['bowling_team'] = deliveries['bowling_team'].replace('Delhi Daredevils', 'Delhi Capitals')
deliveries['batting_team'] = deliveries['batting_team'].replace('Deccan Chargers', 'Sunrisers Hyderabad')
deliveries['bowling_team'] = deliveries['bowling_team'].replace('Deccan Chargers', 'Sunrisers Hyderabad')
deliveries['batting_team'] = deliveries['batting_team'].replace('Kings XI Punjab', 'Punjab Kings')
deliveries['bowling_team'] = deliveries['bowling_team'].replace('Kings XI Punjab', 'Punjab Kings')
deliveries['batting_team'] = deliveries['batting_team'].replace('Rising Pune Supergiants', 'Rising Pune Supergiant')
deliveries['bowling_team'] = deliveries['bowling_team'].replace('Rising Pune Supergiants', 'Rising Pune Supergiant')
deliveries['batting_team'] = deliveries['batting_team'].replace('Royal Challengers Bangalore', 'Royal Challengers Bengaluru')
deliveries['bowling_team'] = deliveries['bowling_team'].replace('Royal Challengers Bangalore', 'Royal Challengers Bengaluru')

In [ ]:
matches["team1"].unique()
# list(matches[matches["method"].isna()]["id"]).index(1082648)

array(['Royal Challengers Bengaluru', 'Punjab Kings', 'Delhi Capitals',
       'Mumbai Indians', 'Kolkata Knight Riders', 'Rajasthan Royals',
       'Sunrisers Hyderabad', 'Chennai Super Kings',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Lucknow Super Giants', 'Gujarat Titans'],
      dtype=object)

In [ ]:
from collections import defaultdict

# Initialize team statistics dictionary
team_stats = defaultdict(lambda: {
    "team_name": "",
    "match_played": 0,
    "matches_won": 0,
    "win_percentage": 0.0,
    "total_batting_run": 0,
    "total_balls_faced": 0,
    "run_rate": 0.0,
    "total_bowling_run": 0,
    "total_overs_bowled": 0.0,
    "economy_rate": 0.0,
    "total_4s": 0,
    "total_6s": 0,
    "highest_score": 0,
    "lowest_score": float("inf"),
    "powerplay_batting_run": 0,
    "powerplay_count": 0,
    "powerplay_wickets": 0,
    "powerplay_bowling_run": 0,
    "powerplay_boundary_count": 0,
    "powerplay_dot_count": 0,
    "average_powerplay_score": 0.0,
    "death_runs": 0,
    "death_count": 0,
    "average_death_score": 0.0
})

match_team_scores = defaultdict(lambda: defaultdict(int))
team_match_runs = defaultdict(lambda: defaultdict(int))

valid_matches = matches[matches["method"].isna()] # Filtering matches that were not affected by the D/L method
# valid_matches = valid_matches[valid_matches["winner"].notna()] # filtering matches that have no result
valid_matches = valid_matches[valid_matches["result"].notna() & (valid_matches["result"] != "no result")]

valid_matches_1 = matches[matches["result"].notna() & (matches["result"] != "no result")]

# Process matches.csv
for _, row in valid_matches_1.iterrows():
    team1, team2, winner = row["team1"], row["team2"], row["winner"]

    for team in [team1, team2]:
        team_stats[team]["team_name"] = team
        team_stats[team]["match_played"] += 1

    if pd.notna(winner):
        team_stats[winner]["matches_won"] += 1

pcount = False
dcount = False

# Process deliveries.csv
for _, row in deliveries.iterrows():

    batting_team, bowling_team = row["batting_team"], row["bowling_team"]
    total_runs, over, is_wicket = row["total_runs"], row["over"], row["is_wicket"]
    extra_type, tr = row["extras_type"], row["batsman_runs"]
    match_id, inning = row["match_id"], row["inning"]

    if match_id in valid_matches["id"].values:
        # if (match_id == 1082648) :
        #   print(match_id)
        match_team_scores[(match_id, inning)][batting_team] += total_runs
        team_match_runs[batting_team][match_id] += total_runs


    # Update batting stats
    team_stats[batting_team]["total_batting_run"] += total_runs
    if pd.isna(extra_type) or extra_type not in ["wides", "noballs"]:
        team_stats[batting_team]["total_balls_faced"] += 1

    # Update boundary stats
    if 4 <= total_runs < 6:
        team_stats[batting_team]["total_4s"] += 1
    elif total_runs >= 6:
        team_stats[batting_team]["total_6s"] += 1

    # Powerplay stats
    if over <= 5:
        pcount = False

        team_stats[batting_team]["powerplay_batting_run"] += total_runs
        if total_runs >= 4:
            team_stats[batting_team]["powerplay_boundary_count"] += 1
        if total_runs == 0:
            team_stats[bowling_team]["powerplay_dot_count"] += 1
        if is_wicket == 1:
            team_stats[bowling_team]["powerplay_wickets"] += 1
        if pd.isna(extra_type) or extra_type in ["wides", "noballs"]:
            team_stats[bowling_team]["powerplay_bowling_run"] += total_runs
        else:
            team_stats[bowling_team]["powerplay_bowling_run"] += tr

    elif over > 5 and pcount == False :
        team_stats[batting_team]["powerplay_count"] += 1
        pcount = True

    #Death over stats
    if over < 16 :
      dcount = False

    elif over == 16 and dcount == False :
      team_stats[batting_team]["death_count"] += 1
      dcount = True

    if over >= 16:
        team_stats[batting_team]["death_runs"] += total_runs


    # Bowling stats
    if pd.isna(extra_type) or extra_type in ["wides", "noballs"]:
        team_stats[bowling_team]["total_bowling_run"] += total_runs
    else:
        team_stats[bowling_team]["total_bowling_run"] += tr

    if pd.isna(extra_type) or extra_type not in ["wides", "noballs"]:
        overs_bowled = team_stats[bowling_team]["total_overs_bowled"]
        if round(overs_bowled % 1, 1) == 0.5:
            team_stats[bowling_team]["total_overs_bowled"] = int(overs_bowled) + 1.0
        else:
            team_stats[bowling_team]["total_overs_bowled"] += 0.1

# Compute final statistics
for team, stats in team_stats.items():
    stats["win_percentage"] = (stats["matches_won"] * 100 / stats["match_played"]) if stats["match_played"] > 0 else 0.0
    stats["run_rate"] = (stats["total_batting_run"] / (stats["total_balls_faced"] / 6)) if stats["total_balls_faced"] > 0 else 0.0
    stats["economy_rate"] = (stats["total_bowling_run"] / stats["total_overs_bowled"]) if stats["total_overs_bowled"] > 0 else 0.0
    stats["average_powerplay_score"] = (stats["powerplay_batting_run"] / stats["powerplay_count"]) if stats["powerplay_count"] > 0 else 0.0
    stats["average_death_score"] = (stats["death_runs"] / stats["death_count"]) if stats["death_count"] > 0 else 0.0

for (match_id, inning), team_scores in match_team_scores.items():
    if match_id in valid_matches["id"].values:
        for team, score in team_scores.items():
            team_stats[team]["team_name"] = team
            team_stats[team]["highest_score"] = max(team_stats[team]["highest_score"], score)
            # team_stats[team]["lowest_score"] = min(team_stats[team]["lowest_score"], score)

for team in team_stats.keys():
    team_stats[team]['lowest_score'] = min(team_match_runs[team].values(), default=float('inf'))

# Convert dictionary to DataFrame and save as CSV
team_stats = pd.DataFrame(team_stats.values())
team_stats.to_csv("team_stat.csv", index=False)
team_stats.head(5)


,team_name,match_played,matches_won,win_percentage,total_batting_run,total_balls_faced,run_rate,total_bowling_run,total_overs_bowled,economy_rate,...,powerplay_batting_run,powerplay_count,powerplay_wickets,powerplay_bowling_run,powerplay_boundary_count,powerplay_dot_count,average_powerplay_score,death_runs,death_count,average_death_score
0,Royal Challengers Bengaluru,252,123,48.809524,40622,28943,8.421104,40000,4831.3,8.279345,...,11795,252,362,11843,1799,4193,46.805556,9028,233,38.746781
1,Kolkata Knight Riders,251,131,52.191235,39331,28385,8.313757,38335,4780.0,8.019874,...,11941,249,337,11718,1870,4189,47.955823,8053,224,35.950893
2,Punjab Kings,246,112,45.528455,39600,28429,8.357663,39243,4687.2,8.372376,...,11795,246,334,11675,1824,4084,47.947154,8274,230,35.973913
3,Chennai Super Kings,237,138,58.227848,38629,27626,8.389705,36540,4588.5,7.963387,...,10991,237,362,11028,1667,4013,46.375527,9061,228,39.741228
4,Delhi Capitals,250,115,46.000000,39196,28708,8.192002,39369,4811.5,8.182272,...,12069,250,359,11608,1890,4158,48.276000,8184,234,34.974359


In [ ]:
print(team_stats[['team_name', 'highest_score', 'lowest_score']])
# print(list(team_match_runs["Kolkata Knight Riders"])[146])

In [ ]:
print(teams[['team_name', 'highest_score', 'lowest_score']])

                      team_name  highest_score  lowest_score
0   Royal Challengers Bengaluru            263            49
1         Kolkata Knight Riders            272            67
2           Chennai Super Kings            246            71
3                  Punjab Kings            262            68
4              Rajasthan Royals            226            58
5                Delhi Capitals            257            66
6                Mumbai Indians            247            68
7           Sunrisers Hyderabad            287            72
8          Kochi Tuskers Kerala            184            74
9                 Pune Warriors            192            99
10       Rising Pune Supergiant            195            78
11                Gujarat Lions            208           104
12         Lucknow Super Giants            257            82
13               Gujarat Titans            233            89


In [ ]:
matches['season_no'] = matches['id'].astype(str).str[:3].astype(int)
deliveries['season_no'] = deliveries['match_id'].astype(str).str[:3].astype(int)

In [ ]:
matches.head()

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,...,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2,season_no
0,335982,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bengaluru,Kolkata Knight Riders,Royal Challengers Bengaluru,...,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen,335
1,335983,2007/08,Chandigarh,2008-04-19,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Punjab Kings,Chennai Super Kings,Chennai Super Kings,...,Chennai Super Kings,runs,33.0,241.0,20.0,N,NaN,MR Benson,SL Shastri,335
2,335984,2007/08,Delhi,2008-04-19,League,MF Maharoof,Feroz Shah Kotla,Delhi Capitals,Rajasthan Royals,Rajasthan Royals,...,Delhi Capitals,wickets,9.0,130.0,20.0,N,NaN,Aleem Dar,GA Pratapkumar,335
3,335985,2007/08,Mumbai,2008-04-20,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Royal Challengers Bengaluru,Mumbai Indians,...,Royal Challengers Bengaluru,wickets,5.0,166.0,20.0,N,NaN,SJ Davis,DJ Harper,335
4,335986,2007/08,Kolkata,2008-04-20,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Sunrisers Hyderabad,Sunrisers Hyderabad,...,Kolkata Knight Riders,wickets,5.0,111.0,20.0,N,NaN,BF Bowden,K Hariharan,335


In [ ]:
matches['season_no'].unique()

array([335, 336, 392, 419, 501, 548, 597, 598, 729, 733, 734, 829, 980,
       981, 108, 113, 117, 118, 121, 123, 125, 130, 131, 135, 137, 142])

In [ ]:
matches['season_no'] = matches['season_no'].replace(335, 1)
deliveries['season_no'] = deliveries['season_no'].replace(335, 1)

matches['season_no'] = matches['season_no'].replace(336, 1)
deliveries['season_no'] = deliveries['season_no'].replace(336, 1)

matches['season_no'] = matches['season_no'].replace(392, 2)
deliveries['season_no'] = deliveries['season_no'].replace(392, 2)

matches['season_no'] = matches['season_no'].replace(419, 3)
deliveries['season_no'] = deliveries['season_no'].replace(419, 3)

matches['season_no'] = matches['season_no'].replace(501, 4)
deliveries['season_no'] = deliveries['season_no'].replace(501, 4)

matches['season_no'] = matches['season_no'].replace(548, 5)
deliveries['season_no'] = deliveries['season_no'].replace(548, 5)

matches['season_no'] = matches['season_no'].replace(597, 6)
deliveries['season_no'] = deliveries['season_no'].replace(597, 6)

matches['season_no'] = matches['season_no'].replace(598, 6)
deliveries['season_no'] = deliveries['season_no'].replace(598, 6)

matches['season_no'] = matches['season_no'].replace(729, 7)
deliveries['season_no'] = deliveries['season_no'].replace(729, 7)

matches['season_no'] = matches['season_no'].replace(733, 7)
deliveries['season_no'] = deliveries['season_no'].replace(733, 7)

matches['season_no'] = matches['season_no'].replace(734, 7)
deliveries['season_no'] = deliveries['season_no'].replace(734, 7)

matches['season_no'] = matches['season_no'].replace(829, 8)
deliveries['season_no'] = deliveries['season_no'].replace(829, 8)

matches['season_no'] = matches['season_no'].replace(980, 9)
deliveries['season_no'] = deliveries['season_no'].replace(980, 9)

matches['season_no'] = matches['season_no'].replace(981, 9)
deliveries['season_no'] = deliveries['season_no'].replace(981, 9)

matches['season_no'] = matches['season_no'].replace(108, 10)
deliveries['season_no'] = deliveries['season_no'].replace(108, 10)

matches['season_no'] = matches['season_no'].replace(113, 11)
deliveries['season_no'] = deliveries['season_no'].replace(113, 11)

matches['season_no'] = matches['season_no'].replace(117, 12)
deliveries['season_no'] = deliveries['season_no'].replace(117, 12)

matches['season_no'] = matches['season_no'].replace(118, 12)
deliveries['season_no'] = deliveries['season_no'].replace(118, 12)

matches['season_no'] = matches['season_no'].replace(121, 13)
deliveries['season_no'] = deliveries['season_no'].replace(121, 13)

matches['season_no'] = matches['season_no'].replace(123, 13)
deliveries['season_no'] = deliveries['season_no'].replace(123, 13)

matches['season_no'] = matches['season_no'].replace(125, 14)
deliveries['season_no'] = deliveries['season_no'].replace(125, 14)

matches['season_no'] = matches['season_no'].replace(130, 15)
deliveries['season_no'] = deliveries['season_no'].replace(130, 15)

matches['season_no'] = matches['season_no'].replace(131, 15)
deliveries['season_no'] = deliveries['season_no'].replace(131, 15)

matches['season_no'] = matches['season_no'].replace(135, 16)
deliveries['season_no'] = deliveries['season_no'].replace(135, 16)

matches['season_no'] = matches['season_no'].replace(137, 16)
deliveries['season_no'] = deliveries['season_no'].replace(137, 16)

matches['season_no'] = matches['season_no'].replace(142, 17)
deliveries['season_no'] = deliveries['season_no'].replace(142, 17)


In [ ]:
matches['season_no'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

In [ ]:
matches.sample(20)

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,...,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2,season_no
466,829721,2015,Ahmedabad,2015-04-14,League,SPD Smith,"Sardar Patel Stadium, Motera",Rajasthan Royals,Mumbai Indians,Mumbai Indians,...,Rajasthan Royals,wickets,7.0,165.0,20.0,N,NaN,AK Chaudhary,SD Fry,8
122,419113,2009/10,Kolkata,2010-03-16,League,MS Dhoni,Eden Gardens,Kolkata Knight Riders,Chennai Super Kings,Chennai Super Kings,...,Chennai Super Kings,runs,55.0,165.0,20.0,N,NaN,HDPK Dharmasena,AM Saheba,3
803,1216499,2020/21,Abu Dhabi,2020-10-28,League,SA Yadav,Sheikh Zayed Stadium,Royal Challengers Bengaluru,Mumbai Indians,Mumbai Indians,...,Mumbai Indians,wickets,5.0,165.0,20.0,N,NaN,UV Gandhe,CB Gaffaney,13
874,1254116,2021,Sharjah,2021-10-13,Qualifier 2,VR Iyer,Sharjah Cricket Stadium,Delhi Capitals,Kolkata Knight Riders,Kolkata Knight Riders,...,Kolkata Knight Riders,wickets,3.0,136.0,20.0,N,NaN,KN Ananthapadmanabhan,MA Gough,14
860,1254102,2021,Dubai,2021-10-01,League,KL Rahul,Dubai International Cricket Stadium,Kolkata Knight Riders,Punjab Kings,Punjab Kings,...,Punjab Kings,wickets,5.0,166.0,20.0,N,NaN,KN Ananthapadmanabhan,RK Illingworth,14
86,392211,2009,Durban,2009-05-05,League,G Gambhir,Kingsmead,Delhi Capitals,Kolkata Knight Riders,Kolkata Knight Riders,...,Delhi Capitals,wickets,9.0,155.0,20.0,N,NaN,GAV Baxter,IL Howell,2
411,729305,2014,Abu Dhabi,2014-04-26,League,PV Tambe,Sheikh Zayed Stadium,Rajasthan Royals,Royal Challengers Bengaluru,Rajasthan Royals,...,Rajasthan Royals,wickets,6.0,71.0,20.0,N,NaN,HDPK Dharmasena,C Shamshuddin,7
592,1082606,2017,Mumbai,2017-04-16,League,N Rana,Wankhede Stadium,Mumbai Indians,Gujarat Lions,Mumbai Indians,...,Mumbai Indians,wickets,6.0,177.0,20.0,N,NaN,A Nand Kishore,S Ravi,10
532,980931,2016,Pune,2016-04-22,League,AB de Villiers,Maharashtra Cricket Association Stadium,Rising Pune Supergiant,Royal Challengers Bengaluru,Rising Pune Supergiant,...,Royal Challengers Bengaluru,runs,13.0,186.0,20.0,N,NaN,CB Gaffaney,VK Sharma,9
1010,1359535,2023,Chennai,2023-05-14,League,RK Singh,"MA Chidambaram Stadium, Chepauk, Chennai",Chennai Super Kings,Kolkata Knight Riders,Chennai Super Kings,...,Kolkata Knight Riders,wickets,6.0,145.0,20.0,N,NaN,Tapan Sharma,Vinod Seshan,16


In [ ]:
matches.head(10)

,id,season,city,date,match_type,player_of_match,venue,team1,team2,toss_winner,...,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2,season_no
0,335982,2007/08,Bangalore,2008-04-18,League,BB McCullum,M Chinnaswamy Stadium,Royal Challengers Bengaluru,Kolkata Knight Riders,Royal Challengers Bengaluru,...,Kolkata Knight Riders,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen,1
1,335983,2007/08,Chandigarh,2008-04-19,League,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",Punjab Kings,Chennai Super Kings,Chennai Super Kings,...,Chennai Super Kings,runs,33.0,241.0,20.0,N,NaN,MR Benson,SL Shastri,1
2,335984,2007/08,Delhi,2008-04-19,League,MF Maharoof,Feroz Shah Kotla,Delhi Capitals,Rajasthan Royals,Rajasthan Royals,...,Delhi Capitals,wickets,9.0,130.0,20.0,N,NaN,Aleem Dar,GA Pratapkumar,1
3,335985,2007/08,Mumbai,2008-04-20,League,MV Boucher,Wankhede Stadium,Mumbai Indians,Royal Challengers Bengaluru,Mumbai Indians,...,Royal Challengers Bengaluru,wickets,5.0,166.0,20.0,N,NaN,SJ Davis,DJ Harper,1
4,335986,2007/08,Kolkata,2008-04-20,League,DJ Hussey,Eden Gardens,Kolkata Knight Riders,Sunrisers Hyderabad,Sunrisers Hyderabad,...,Kolkata Knight Riders,wickets,5.0,111.0,20.0,N,NaN,BF Bowden,K Hariharan,1
5,335987,2007/08,Jaipur,2008-04-21,League,SR Watson,Sawai Mansingh Stadium,Rajasthan Royals,Punjab Kings,Punjab Kings,...,Rajasthan Royals,wickets,6.0,167.0,20.0,N,NaN,Aleem Dar,RB Tiffin,1
6,335988,2007/08,Hyderabad,2008-04-22,League,V Sehwag,"Rajiv Gandhi International Stadium, Uppal",Sunrisers Hyderabad,Delhi Capitals,Sunrisers Hyderabad,...,Delhi Capitals,wickets,9.0,143.0,20.0,N,NaN,IL Howell,AM Saheba,1
7,335989,2007/08,Chennai,2008-04-23,League,ML Hayden,"MA Chidambaram Stadium, Chepauk",Chennai Super Kings,Mumbai Indians,Mumbai Indians,...,Chennai Super Kings,runs,6.0,209.0,20.0,N,NaN,DJ Harper,GA Pratapkumar,1
8,335990,2007/08,Hyderabad,2008-04-24,League,YK Pathan,"Rajiv Gandhi International Stadium, Uppal",Sunrisers Hyderabad,Rajasthan Royals,Rajasthan Royals,...,Rajasthan Royals,wickets,3.0,215.0,20.0,N,NaN,Asad Rauf,MR Benson,1
9,335991,2007/08,Chandigarh,2008-04-25,League,KC Sangakkara,"Punjab Cricket Association Stadium, Mohali",Punjab Kings,Mumbai Indians,Mumbai Indians,...,Punjab Kings,runs,66.0,183.0,20.0,N,NaN,Aleem Dar,AM Saheba,1


In [ ]:
deliveries['dismissal_kind'].unique()

array([nan, 'caught', 'bowled', 'run out', 'lbw', 'retired hurt',
       'stumped', 'caught and bowled', 'hit wicket',
       'obstructing the field', 'retired out'], dtype=object)

In [ ]:
deliveries.head()

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder,season_no
0,335982,1,Kolkata Knight Riders,Royal Challengers Bengaluru,0,1,SC Ganguly,P Kumar,BB McCullum,0,1,1,legbyes,0,NaN,NaN,NaN,1
1,335982,1,Kolkata Knight Riders,Royal Challengers Bengaluru,0,2,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN,1
2,335982,1,Kolkata Knight Riders,Royal Challengers Bengaluru,0,3,BB McCullum,P Kumar,SC Ganguly,0,1,1,wides,0,NaN,NaN,NaN,1
3,335982,1,Kolkata Knight Riders,Royal Challengers Bengaluru,0,4,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN,1
4,335982,1,Kolkata Knight Riders,Royal Challengers Bengaluru,0,5,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN,1


In [ ]:
season_stats = []

# Get unique seasons
unique_seasons = sorted(matches['season_no'].unique())

for season in tqdm(unique_seasons, total=len(unique_seasons)):
    # Filter deliveries and matches for the current season
    season_deliveries = deliveries[deliveries['season_no'] == season]
    season_matches = matches[matches['season_no'] == season]

    sy = season_matches['season'].iloc[0] if not season_matches.empty else None

    total_run_in_season = season_deliveries['total_runs'].sum()
    total_number_of_innings = season_matches['id'].nunique() * 2
    average_run_per_match = total_run_in_season / total_number_of_innings if total_number_of_innings != 0 else 0

    # Targets 200+
    targets_200 = 0
    for match_id in season_matches['id'].unique():
        first_innings_runs = season_deliveries[(season_deliveries['match_id'] == match_id) &
                                               (season_deliveries['inning'] == 1)]['total_runs'].sum()
        if first_innings_runs >= 200:
            targets_200 += 1

    # Orange Cap: Highest run scorer
    batsman_runs = season_deliveries.groupby('batter')['batsman_runs'].sum()
    orange_cap_holder = batsman_runs.idxmax() if not batsman_runs.empty else None
    orange_cap_run = batsman_runs.max() if not batsman_runs.empty else None

    # Purple Cap: Highest wicket taker
    season_deliveries_filtered = season_deliveries[~season_deliveries['dismissal_kind'].isin(['run out', 'retired hurt', 'obstructing the field', 'retired out'])]
    bowler_wickets = season_deliveries_filtered.groupby('bowler')['is_wicket'].sum()
    purple_cap_holder = bowler_wickets.idxmax() if not bowler_wickets.empty else None
    purple_cap_wicket = bowler_wickets.max() if not bowler_wickets.empty else None

    # Append to list
    season_stats.append({
        'Season no': season,
        'Year': sy,
        'total_run_in_season': total_run_in_season,
        'total_number_of_innings': total_number_of_innings,
        'average_run_per_match': round(average_run_per_match, 2),
        'targets_200+': targets_200,
        'orange_cap_holder': orange_cap_holder,
        'orange_cap_run': orange_cap_run,
        'purple_cap_holder': purple_cap_holder,
        'purple_cap_wicket': purple_cap_wicket
    })

season_stats_df = pd.DataFrame(season_stats)

output_path = 'seasonal_stat.csv'
season_stats_df.to_csv(output_path, index=False)

season_stats_df.head()

100%|██████████| 17/17 [00:00<00:00, 21.36it/s]


,Season no,Year,total_run_in_season,total_number_of_innings,average_run_per_match,targets_200+,orange_cap_holder,orange_cap_run,purple_cap_holder,purple_cap_wicket
0,1,2007/08,17937,116,154.63,7,SE Marsh,616,Sohail Tanvir,22
1,2,2009,16353,114,143.45,1,ML Hayden,572,RP Singh,23
2,3,2009/10,18883,120,157.36,5,SR Tendulkar,618,PP Ojha,21
3,4,2011,21154,146,144.89,4,CH Gayle,608,SL Malinga,28
4,5,2012,22453,148,151.71,4,CH Gayle,733,M Morkel,25


In [ ]:
# Merge deliveries with matches to get season information
deliveries = deliveries.merge(matches[["id", "season_no"]], left_on="match_id", right_on="id", how="left")

# Filter only wicket-taking deliveries (excluding run-outs)
valid_dismissals = ["bowled", "caught", "lbw", "stumped", "caught and bowled", "hit wicket"]
valid_deliveries = deliveries[~deliveries["extras_type"].isin(["wides", "noballs"])]
wickets_df = deliveries[deliveries["dismissal_kind"].isin(valid_dismissals)]

if 'season_no' not in wickets_df.columns:
    wickets_df = wickets_df.merge(matches[['id', 'season_no']], left_on='match_id', right_on='id', how='left')

bowler_wickets = wickets_df.groupby(["season_no", "bowler"]).size().reset_index(name="wickets")
runs_conceded = valid_deliveries.groupby(["season_no", "bowler"])["total_runs"].sum().reset_index()
balls_bowled = valid_deliveries.groupby(["season_no", "bowler"]).size().reset_index(name="balls")

bowler_stats = runs_conceded.merge(balls_bowled, on=["season_no", "bowler"])
bowler_stats["overs"] = bowler_stats["balls"] / 6
bowler_stats["economy_rate"] = bowler_stats["total_runs"] / bowler_stats["overs"]

# Get top 10 bowlers per season
top_bowlers_per_season = bowler_wickets.groupby("season_no").apply(lambda x: x.nlargest(10, "wickets")).reset_index(drop=True)

top_bowlers_stats = top_bowlers_per_season.merge(bowler_stats, on=["season_no", "bowler"])

top_bowlers_stats = top_bowlers_stats[["season_no", "bowler", "wickets", "economy_rate"]]

top_bowlers_stats.head(20)  # Show top 10 bowlers for the first 2 seasons as an example


<ipython-input-35-beb68fcb9665>:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_bowlers_per_season = bowler_wickets.groupby("season_no").apply(lambda x: x.nlargest(10, "wickets")).reset_index(drop=True)


,season_no,bowler,wickets,economy_rate
0,1,Sohail Tanvir,22,6.194332
1,1,S Sreesanth,19,8.032573
2,1,SK Warne,19,7.692308
3,1,JA Morkel,17,8.020833
4,1,MS Gony,17,7.400000
5,1,PP Chawla,17,8.156584
6,1,SR Watson,17,6.886154
7,1,VY Mahesh,16,8.324111
8,1,IK Pathan,15,6.452830
9,1,MF Maharoof,15,6.777778


In [ ]:
top_bowlers_stats.to_csv('top_bowlers_stats.csv', index=False)

In [ ]:
team_season_stats = deliveries.groupby(['batting_team', 'season_no'])['total_runs'].sum().reset_index()
team_season_matches = deliveries.groupby(['batting_team', 'season_no'])['match_id'].nunique().reset_index()

# Renaming columns for our clarity
team_season_stats = team_season_stats.rename(columns={'batting_team': 'team_name', 'total_runs': 'total_runs'})
team_season_matches = team_season_matches.rename(columns={'batting_team': 'team_name', 'match_id': 'total_matches'})

team_season_stats = pd.merge(team_season_stats, team_season_matches, on=['team_name', 'season_no'])

team_season_stats['average_score'] = team_season_stats['total_runs'] / team_season_stats['total_matches']

team_season_stats

,team_name,season_no,total_runs,total_matches,average_score
0,Chennai Super Kings,1,2520,16,157.500000
1,Chennai Super Kings,2,2231,14,159.357143
2,Chennai Super Kings,3,2604,16,162.750000
3,Chennai Super Kings,4,2560,16,160.000000
4,Chennai Super Kings,5,2831,18,157.277778
...,...,...,...,...,...
141,Sunrisers Hyderabad,13,2531,16,158.187500
142,Sunrisers Hyderabad,14,2084,14,148.857143
143,Sunrisers Hyderabad,15,2197,14,156.928571
144,Sunrisers Hyderabad,16,2376,14,169.714286


In [ ]:
team_season_stats.to_csv('team_season_stats.csv', index=False)